In [1]:
import pandas as pd
import numpy as np

# Cargar el dataset limpio
df = pd.read_csv("online_retail_limpio.csv")

In [ ]:
# Convierto de nuevo fechas a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Elimino filas con valores nulos esenciales
df.dropna(subset=['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice', 'InvoiceDate'], inplace=True)

# Añado ls columna 'TotalSales' = Quantity * UnitPrice
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

# Creo la columna 'Month' para agrupaciones por mes
df['Month'] = df['InvoiceDate'].dt.to_period('M')

# Creo el dataframe con tasa de rotación por producto por mes
rotation_df = df.groupby(['StockCode', 'Month']).agg({
    'Quantity': 'sum',
    'TotalSales': 'sum'
}).reset_index()

# Los productos con baja rotación estimo que son aquellos vendieron menos de 5 unidades en un mes
low_rotation = rotation_df[rotation_df['Quantity'] < 5]

# Cancelaciones: InvoiceNo que empiezan por 'C'
df['Cancelled'] = df['InvoiceNo'].astype(str).str.startswith('C')
cancelled_by_country = df[df['Cancelled']].groupby('Country').agg({
    'InvoiceNo': 'nunique'
}).rename(columns={'InvoiceNo': 'NumCancelledInvoices'}).reset_index()

# Inventario inmovilizado: productos no vendidos en los últimos 3 meses
last_month = df['InvoiceDate'].max().to_period('M')
cutoff_month = (last_month - 3).start_time
recent_sales = df[df['InvoiceDate'] >= cutoff_month]
recent_products = recent_sales['StockCode'].unique()
immobile_inventory = df[~df['StockCode'].isin(recent_products)]

# Coste de inventario inmovilizado
immobile_cost = immobile_inventory.groupby('StockCode').agg({
    'Quantity': 'sum',
    'UnitPrice': 'mean'
})
immobile_cost['EstimatedValue'] = immobile_cost['Quantity'] * immobile_cost['UnitPrice']
immobile_cost_total = immobile_cost['EstimatedValue'].sum()


In [4]:
# Exportar productos con baja rotación
low_rotation.to_csv("productos_baja_rotacion.csv", index=False)

# Exportar inventario inmovilizado con su valor estimado
immobile_cost.reset_index().to_csv("inventario_inmovilizado.csv", index=False)

# Exportar tasa de rotación mensual por producto
rotation_df.to_csv("tasa_rotacion_mensual.csv", index=False)

In [ ]:
# Agrupo por producto y país para obtener métricas por combinación
df_alertas = df.groupby(["Description", "Country"]).agg({
    "Quantity": "sum",
    "UnitPrice": "mean"
}).reset_index()

# Renombro las columnas para que sean mucho más claras
df_alertas.columns = ["Product", "Country", "Quantity", "Avg_UnitPrice"]

# Aplico las reglas de alerta con tres niveles de urgencia
def clasificar_alerta(row):
    if row["Quantity"] > 300 and row["Avg_UnitPrice"] > 10:
        return "CRITICA"
    elif row["Quantity"] > 300:
        return "MEDIA"
    else:
        return "OK"

df_alertas["AlertFlag"] = df_alertas.apply(clasificar_alerta, axis=1)